In [2]:
using TensorKit
import TensorKit.×
include("./iPEPS/iPEPS.jl")
include("./iPEPS/swap_gate.jl")
include("./iPEPS/util.jl")
@show pkgversion(TensorKit)
@show pkgdir(TensorKit)

pkgversion(TensorKit) = v"0.12.1"
pkgdir(TensorKit) = "C:\\Users\\quxin\\.julia\\packages\\TensorKit\\Xkabb"


"C:\\Users\\quxin\\.julia\\packages\\TensorKit\\Xkabb"

In [15]:
# pspace = GradedSpace{fSU₂}((-1, 0) => 1, (0, 1 // 2) => 1, (1, 0) => 1)
pspace = GradedSpace{fSU₂}((0 => 2), (1//2 => 1))
aspacel = aspacer = aspacet = aspaceb = pspace
@show pspace
nothing

pspace = Vect[FermionSpin]((0, 0)=>2, (1/2, 1)=>1)


In [ ]:
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# pspace = Rep[ℤ₂]((1 => 1), (0 => 2))
aspacel = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspacer = Rep[ℤ₂×SU₂]((0, 0) => 2, (0, 1) => 2)
aspacer = aspacel
aspacet = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspaceb = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 3 // 2) => 2)
aspaceb = aspacet

# A = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
# Ms = [A, A, A, A]
# ipeps = iPEPS(Ms, 2, 2)
# Abar = bar(A)
# @show space(Abar)
# gate1 = swap_gate(space(Abar)[4], space(Abar)[5])
# @show space(gate1)
# gate2 = swap_gate(space(Abar)[1], space(Abar)[2])
# @show space(gate2)
# gate3 = swap_gate(space(A)[4], space(A)[5])
# @show space(gate3)
# gate4 = swap_gate(space(A)[1], space(A)[2])
# @show space(gate4)
# gate5 = swap_gate(space(A)[1], space(Abar)[2])
# @show space(gate5)
@assert pspace isa ElementarySpace
@show fieldnames(GradedSpace)

In [6]:
A = id(pspace)
@show A
exp!(A)
@show A + A
nothing

A = TensorMap(Vect[FermionSpin]((0, 0)=>2, (1/2, 1)=>1) ← Vect[FermionSpin]((0, 0)=>2, (1/2, 1)=>1)):
* Data for fusiontree FusionTree{FermionSpin}(((0, 0),), (0, 0), (false,), ()) ← FusionTree{FermionSpin}(((0, 0),), (0, 0), (false,), ()):
 1.0  0.0
 0.0  1.0
* Data for fusiontree FusionTree{FermionSpin}(((1/2, 1),), (1/2, 1), (false,), ()) ← FusionTree{FermionSpin}(((1/2, 1),), (1/2, 1), (false,), ()):
 1.0

A + A = TensorMap(Vect[FermionSpin]((0, 0)=>2, (1/2, 1)=>1) ← Vect[FermionSpin]((0, 0)=>2, (1/2, 1)=>1)):
* Data for fusiontree FusionTree{FermionSpin}(((0, 0),), (0, 0), (false,), ()) ← FusionTree{FermionSpin}(((0, 0),), (0, 0), (false,), ()):
 5.43656365691809  0.0
 0.0               5.43656365691809
* Data for fusiontree FusionTree{FermionSpin}(((1/2, 1),), (1/2, 1), (false,), ()) ← FusionTree{FermionSpin}(((1/2, 1),), (1/2, 1), (false,), ()):
 5.43656365691809



In [11]:
ipeps = iPEPSΓΛ(pspace, aspacel, aspacet, 2, 2; dtype=ComplexF64)

gateNN = TensorMap(randn, ComplexF64, pspace ⊗ pspace, pspace ⊗ pspace)

nothing

In [ ]:


nothing

In [ ]:
             
nothing